## Кривоногов Н.В., NLP, практическое задание № 2

### Задание 1.

**Задание**: обучите три классификатора:

1) на токенах с высокой частотой

2) на токенах со средней частотой

3) на токенах с низкой частотой


Сравните полученные результаты, оцените какие токены наиболее важные для классификации.


### Задание 2.

найти фичи с наибольшей значимостью, и вывести их


### Задание 3.

1) сравнить count/tf-idf/hashing векторайзеры/полносвязанную сетку (построить classification_report)

2) подобрать оптимальный размер для hashing векторайзера

3) убедиться что для сетки нет переобучения

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [2]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
my_stopwords = stopwords.words('russian')
noise = stopwords.words('russian')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
positive = pd.read_csv('/content/positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('/content/negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = positive.append(negative)

<ipython-input-3-cd2f27a6c2f9>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = positive.append(negative)


In [4]:
df

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive
...,...,...
111918,Но не каждый хочет что то исправлять:( http://...,negative
111919,скучаю так :-( только @taaannyaaa вправляет мо...,negative
111920,"Вот и в школу, в говно это идти уже надо(",negative
111921,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",negative


In [5]:
X_train, X_test, y_train, y_test = train_test_split(df.text, df.label)

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [7]:
def get_tokens_importance(maxdf, mindf):

    tfidf_vect = TfidfVectorizer(max_df=maxdf, # не берем слова, которые выше трешхолда
                                 min_df=mindf, # не берем слова, которые ниже трешхолда
#                                  max_features=1000,
#                                  stop_words=noise,
                                 )

    bow = tfidf_vect.fit_transform(X_train)
    clf = LogisticRegression(random_state=42)
    clf.fit(bow, y_train)

    pred = clf.predict(tfidf_vect.transform(X_test))
    print(f'max_df= {maxdf} min_df= {mindf}')
    print(classification_report(pred, y_test))

In [8]:
# Берем все токены: f1-score получается низкая:

get_tokens_importance(1, 1)

max_df= 1 min_df= 1
              precision    recall  f1-score   support

    negative       0.89      0.53      0.66     47522
    positive       0.22      0.68      0.33      9187

    accuracy                           0.55     56709
   macro avg       0.56      0.60      0.50     56709
weighted avg       0.78      0.55      0.61     56709



In [9]:
# Берем часто встречающиеся токены:

get_tokens_importance(1.0, 0.285)

max_df= 1.0 min_df= 0.285
              precision    recall  f1-score   support

    negative       0.35      0.60      0.44     16342
    positive       0.77      0.55      0.64     40367

    accuracy                           0.56     56709
   macro avg       0.56      0.57      0.54     56709
weighted avg       0.65      0.56      0.58     56709



In [10]:
# Берем средне встречающиеся токены:

get_tokens_importance(0.7, 0.15)

max_df= 0.7 min_df= 0.15
              precision    recall  f1-score   support

    negative       0.35      0.60      0.44     16100
    positive       0.78      0.55      0.64     40609

    accuracy                           0.56     56709
   macro avg       0.56      0.58      0.54     56709
weighted avg       0.65      0.56      0.59     56709



In [11]:
# Берем редко встречающиеся токены:

get_tokens_importance(0.3, 1)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


max_df= 0.3 min_df= 1
              precision    recall  f1-score   support

    negative       0.73      0.76      0.75     26798
    positive       0.78      0.75      0.76     29911

    accuracy                           0.76     56709
   macro avg       0.76      0.76      0.76     56709
weighted avg       0.76      0.76      0.76     56709



Судя по получившимся метрикам, низко частотные (редко встречающиеся) токены наиболее важны для классификации.

In [12]:
from collections import Counter
from nltk.tokenize import word_tokenize

In [13]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [14]:
corpus = [token for tweet in df.text for token in word_tokenize(tweet)]
print(len(corpus))
corpus[:10]

4027088


['@', 'first_timee', 'хоть', 'я', 'и', 'школота', ',', 'но', 'поверь', ',']

In [15]:
freq_dict = Counter(corpus)
freq_dict_sorted= sorted(freq_dict.items(), key=lambda x: -x[1])
list(freq_dict_sorted)[:10]

[('(', 212404),
 (')', 194005),
 (',', 188295),
 (':', 177675),
 ('@', 149978),
 ('не', 69472),
 ('!', 66923),
 ('.', 57623),
 ('и', 55166),
 ('в', 52902)]

In [16]:
cool_token = '!'
pred = ['positive' if cool_token in tweet else 'negative' for tweet in X_test]
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.87      0.50      0.64     48206
    positive       0.17      0.58      0.26      8503

    accuracy                           0.52     56709
   macro avg       0.52      0.54      0.45     56709
weighted avg       0.77      0.52      0.58     56709



In [17]:
cool_token = ':'
pred = ['positive' if cool_token in tweet else 'negative' for tweet in X_test]
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.48      0.54      0.51     24839
    positive       0.61      0.55      0.58     31870

    accuracy                           0.55     56709
   macro avg       0.55      0.55      0.54     56709
weighted avg       0.55      0.55      0.55     56709



In [18]:
cool_token = '@'
pred = ['positive' if cool_token in tweet else 'negative' for tweet in X_test]
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.57      0.56      0.57     28340
    positive       0.57      0.58      0.57     28369

    accuracy                           0.57     56709
   macro avg       0.57      0.57      0.57     56709
weighted avg       0.57      0.57      0.57     56709



In [19]:
from sklearn.feature_extraction.text import CountVectorizer

In [20]:
%%time

count_vect = CountVectorizer()
bow = count_vect.fit_transform(X_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)

pred = clf.predict(count_vect.transform(X_test))
print(classification_report(pred, y_test))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

    negative       0.76      0.76      0.76     28200
    positive       0.76      0.77      0.76     28509

    accuracy                           0.76     56709
   macro avg       0.76      0.76      0.76     56709
weighted avg       0.76      0.76      0.76     56709

CPU times: user 16.8 s, sys: 9.82 s, total: 26.6 s
Wall time: 18.1 s


In [21]:
%%time

tfidf_vect = TfidfVectorizer()
bow = tfidf_vect.fit_transform(X_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)

pred = clf.predict(tfidf_vect.transform(X_test))
print(classification_report(pred, y_test))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

    negative       0.73      0.76      0.75     26798
    positive       0.78      0.75      0.76     29911

    accuracy                           0.76     56709
   macro avg       0.76      0.76      0.76     56709
weighted avg       0.76      0.76      0.76     56709

CPU times: user 16.3 s, sys: 11.1 s, total: 27.4 s
Wall time: 20.2 s


In [22]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import LogisticRegression

In [23]:
%%time

feature_sizes = [100, 1000, 10000, 100000, 1000000]

for feature_size in feature_sizes:

    h_vect = HashingVectorizer(n_features=feature_size)
    h_vect.fit(X_train)

    xtrain_count =  h_vect.transform(X_train)
    xtest_count =  h_vect.transform(X_test)

    classifier = LogisticRegression()
    classifier.fit(xtrain_count, y_train)
    predictions = classifier.predict(xtest_count)
    #predictions
    print(f'Длина hash-вектора: {feature_size}')
    print(classification_report(predictions, y_test))

Длина hash-вектора: 100
              precision    recall  f1-score   support

    negative       0.58      0.59      0.58     27341
    positive       0.61      0.60      0.60     29368

    accuracy                           0.59     56709
   macro avg       0.59      0.59      0.59     56709
weighted avg       0.59      0.59      0.59     56709

Длина hash-вектора: 1000
              precision    recall  f1-score   support

    negative       0.63      0.65      0.64     27371
    positive       0.66      0.65      0.66     29338

    accuracy                           0.65     56709
   macro avg       0.65      0.65      0.65     56709
weighted avg       0.65      0.65      0.65     56709



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Длина hash-вектора: 10000
              precision    recall  f1-score   support

    negative       0.70      0.71      0.71     27304
    positive       0.73      0.71      0.72     29405

    accuracy                           0.71     56709
   macro avg       0.71      0.71      0.71     56709
weighted avg       0.71      0.71      0.71     56709



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Длина hash-вектора: 100000
              precision    recall  f1-score   support

    negative       0.72      0.75      0.73     26837
    positive       0.77      0.74      0.75     29872

    accuracy                           0.74     56709
   macro avg       0.74      0.74      0.74     56709
weighted avg       0.74      0.74      0.74     56709



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Длина hash-вектора: 1000000
              precision    recall  f1-score   support

    negative       0.72      0.75      0.74     26737
    positive       0.77      0.74      0.76     29972

    accuracy                           0.75     56709
   macro avg       0.75      0.75      0.75     56709
weighted avg       0.75      0.75      0.75     56709

CPU times: user 1min 2s, sys: 28.7 s, total: 1min 30s
Wall time: 1min 5s


Чем больше длина hash-вектора, тем выше метрики.

In [24]:
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, Conv1D, GRU, LSTM, Dropout
from tensorflow.keras.layers import TextVectorization
from sklearn.preprocessing import LabelEncoder

In [25]:
# приведем y к 0,1:

encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)

In [26]:
train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train))
valid_data = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [27]:
train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

In [28]:
for raw in train_data.take(1):
    pass

In [29]:
raw

(<tf.Tensor: shape=(16,), dtype=string, numpy=
 array([b'\xd0\x9d\xd0\xb5 \xd0\xbe\xd1\x88\xd0\xb8\xd0\xb1\xd0\xba\xd1\x83 \xd0\xbb\xd0\xb8 \xd1\x8f \xd1\x81\xd0\xbe\xd0\xb2\xd0\xb5\xd1\x80\xd1\x88\xd0\xb8\xd0\xbb\xd0\xb0.. \xd1\x87\xd1\x82\xd0\xbe-\xd1\x82\xd0\xbe \xd0\xbc\xd0\xbd\xd0\xb5 \xd0\xbd\xd0\xb5 \xd0\xbd\xd1\x80\xd0\xb0\xd0\xb2\xd0\xb8\xd1\x82\xd1\x81\xd1\x8f \xd1\x8d\xd1\x82\xd0\xbe \xd0\xb2\xd1\x81\xd0\xb5...((((',
        b'\xd0\x9f\xd0\xbe\xd1\x81\xd0\xbb\xd0\xb5\xd0\xb4\xd0\xbd\xd1\x8f\xd1\x8f \xd0\xbf\xd1\x8f\xd1\x82\xd0\xbd\xd0\xb8\xd1\x86\xd0\xb0 \xd1\x82\xd1\x80\xd0\xb8\xd0\xbd\xd0\xb0\xd0\xb4\xd1\x86\xd0\xb0\xd1\x82\xd0\xbe\xd0\xb5 \xd1\x82\xd1\x80\xd0\xb8\xd0\xbd\xd0\xb0\xd0\xb4\xd1\x86\xd0\xb0\xd1\x82\xd0\xbe\xd0\xb3\xd0\xbe \xd0\xb3\xd0\xbe\xd0\xb4\xd0\xb0)\xd0\xa1\xd1\x83\xd0\xb4\xd1\x8f \xd0\xbf\xd0\xbe \xd0\xb4\xd0\xb0\xd1\x82\xd0\xb5 \xd0\xb8 \xd1\x82\xd0\xbe\xd0\xbc\xd1\x83,\xd1\x87\xd1\x82\xd0\xbe \xd1\x8f \xd1\x81\xd0\xbd\xd0\xbe\xd0\xb2\xd0\xb0 \xd1\x81\

In [30]:
def custom_standardization(input_data):

    return input_data

vocab_size = 10000 # отбираем 10000 вместо 31000
seq_len = 100 # сколько токенов в тексте проходят по всем текстам
# и выбирают максимум или берут квантильное, могут быть выбросы по длинам

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int', # каждый токен будем переводить в индекс относительно нашего словаря vocab_size
    output_sequence_length=seq_len)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

embedding_dim=200 # 200-мерные вектора будут у каждого токена

In [31]:
class myNet(tf.keras.Model):
    def __init__(self):
        super(myNet, self).__init__()
        self.emb = Embedding(vocab_size, embedding_dim, name="embedding")
        self.conv1 = Conv1D(200, (3))
        self.conv2 = Conv1D(200, (3))
        self.gPool = GlobalAveragePooling1D()
        self.fc1 = Dense(100, activation='relu')
        self.fc2 = Dense(1)


    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        x1 = self.conv1(x)
        x = self.conv2(x)
        x = self.gPool((x + x1)/2)
        x = self.fc1(x)
        return self.fc2(x)

In [32]:
model = myNet()

In [33]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [34]:
model.fit(train_data, validation_data=valid_data, epochs=3)

Epoch 1/3
10633/10633 [==============================] - 113s 10ms/step - loss: 0.3892 - accuracy: 0.7751 - val_loss: 0.3840 - val_accuracy: 0.7568
Epoch 2/3
10633/10633 [==============================] - 81s 8ms/step - loss: 0.3257 - accuracy: 0.8190 - val_loss: 0.3865 - val_accuracy: 0.7773
Epoch 3/3
10633/10633 [==============================] - 82s 8ms/step - loss: 0.3036 - accuracy: 0.8347 - val_loss: 0.4089 - val_accuracy: 0.7799


In [35]:
preds = model.predict(valid_data)

3545/3545 [==============================] - 9s 3ms/step


In [36]:
preds

array([[-2.6517942 ],
       [-1.3252225 ],
       [ 1.5217838 ],
       ...,
       [-4.7884526 ],
       [-1.0737562 ],
       [ 0.16081455]], dtype=float32)

In [37]:
# приведение preds в бинарный вид:

preds = [int(i[0]>0) for i in preds]

In [38]:
print(classification_report(preds, y_test))

              precision    recall  f1-score   support

           0       0.91      0.74      0.82     34239
           1       0.69      0.89      0.78     22470

    accuracy                           0.80     56709
   macro avg       0.80      0.82      0.80     56709
weighted avg       0.83      0.80      0.80     56709



У полносвязанной сети лучшие результаты по метрикам.